In [ ]:
# If necessary, install the library
# pip install basedosdados

In [ ]:
# Import the necessary libraries
import basedosdados as bd
import pandas as pd

# **Localização de chamados do 1746**
## **1. Quantos chamados foram abertos no dia 01/04/2023?**

In [59]:
# Define the billing project ID to be used for executing queries in BigQuery
# This project ID will be 'charged' for the queries executed in the datario dataset
billing_project_id = 'datario-project'

In [66]:
def calls_by_date(date: str):
    '''
    Queries the total number of calls opened on a specific date.

    Parameters:
    date (str): The date for which to retrieve the total number of calls, formatted as 'YYYY-MM-DD'.

    Returns:
    DataFrame: A DataFrame containing the total number of calls for the specified date.
    '''
    
    query1 = f'''
    SELECT COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) = '{date}';
    '''
    df1 = bd.read_sql(query1, billing_project_id)
    return df1

In [67]:
# Retrieve the total number of calls for April 1, 2023
answer1 = calls_by_date('2023-04-01')

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.11rows/s]


In [68]:
# Display the result
display(answer1)

,total_chamados
0,1756


## **2. Qual o tipo de chamado que mais teve chamados abertos no dia 01/04/2023?**

In [70]:
def most_frequent_type(date: str):
    '''
    Queries the most frequently opened type of call for a specific date.

    Parameters:
    date (str): The date for which to retrieve the most frequent call type, formatted as 'YYYY-MM-DD'.

    Returns:
    DataFrame: A DataFrame containing the most frequently opened call type and its count.
    '''
    
    query2 = f'''
    SELECT tipo, COUNT(*) AS type_total
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) = '{date}'
    GROUP BY tipo
    ORDER BY type_total DESC
    LIMIT 1;
    '''
    df2 = bd.read_sql(query2, billing_project_id)
    return df2

In [71]:
# Retrieve the most frequent type of call for April 1, 2023
answer2 = most_frequent_type('2023-04-01')

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.17rows/s]


In [72]:
# Display the result
display(answer2)

,tipo,type_total
0,Estacionamento irregular,366


## **3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?**

In [73]:
def neighborhoods_w_most_calls(date: str):
    '''
    Queries the neighborhoods with the most calls on a specific date.

    Parameters:
    date (str): The date for which to retrieve the top neighborhoods by call volume, formatted as 'YYYY-MM-DD'.

    Returns:
    DataFrame: A DataFrame containing the top 3 neighborhoods with the highest number of calls for the specified date.
    '''
    
    query3 = f'''
    SELECT b.nome, 
    COUNT(*) AS total_chamados_bairro
    FROM `datario.adm_central_atendimento_1746.chamado` AS c
    INNER JOIN `datario.dados_mestres.bairro` AS b
    ON c.id_bairro = b.id_bairro
    WHERE DATE(c.data_inicio) = '{date}'
    GROUP BY b.nome
    ORDER BY total_chamados_bairro DESC
    LIMIT 3;
    '''
    df3 = bd.read_sql(query3, billing_project_id)
    return df3

In [74]:
# Retrieve the top 3 neighborhoods with the highest number of calls for April 1, 2023
answer3 = neighborhoods_w_most_calls('2023-04-01')

Downloading: 100%|██████████| 3/3 [00:00<00:00,  8.14rows/s]


In [75]:
# Display the result
display(answer3)

,nome,total_chamados_bairro
0,Campo Grande,113
1,Tijuca,89
2,Barra da Tijuca,59


## **4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?**

In [76]:
def subprefecture_with_most_calls(date: str):
    '''
    Query to find the subprefecture with the highest number of calls on a specific date.

    Parameters:
    date (str): The date for which to find the subprefecture with the most calls (in 'YYYY-MM-DD' format).

    Returns:
    DataFrame: A DataFrame containing the subprefecture with the highest number of calls and the total number of calls.
    '''
    
    query4 = f'''
    SELECT 
        b.subprefeitura AS subprefeitura_name,
        COUNT(*) AS total_chamados_subprefeitura
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    LEFT JOIN 
        `datario.dados_mestres.bairro` AS b
    ON 
        c.id_bairro = b.id_bairro
    WHERE 
        DATE(c.data_inicio) = '{date}'
    GROUP BY 
        b.subprefeitura
    ORDER BY 
        total_chamados_subprefeitura DESC
    LIMIT 1;
    '''
    df4 = bd.read_sql(query4, billing_project_id)
    return df4

In [77]:
# Retrieve the subprefecture with the highest number of calls for April 1, 2023
answer4 = subprefecture_with_most_calls('2023-04-01')

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.43rows/s]


In [78]:
# Display the result
display(answer4)

,subprefeitura_name,total_chamados_subprefeitura
0,Zona Norte,510


## **5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [79]:
def calls_without_assigned_neighborhood_or_subprefecture(date: str):
    '''
    Queries the number of calls on a specific date that are not associated with a neighborhood
    or subprefecture in the neighborhood table.

    Args:
        date (str): The date to filter the calls in 'YYYY-MM-DD' format.

    Returns:
        pandas.DataFrame: A DataFrame containing the count of calls without assigned 
        neighborhood or subprefecture for the specified date, along with their details if available.
        
    Notes:
        This function checks for calls that do not have a corresponding entry in the
        neighborhood table, which might be due to data entry issues or missing records.
    '''
    
    query5 = f'''
    SELECT 
        c.id_chamado,
        c.data_inicio,
        c.id_bairro,
        b.id_bairro AS bairro_associado
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    LEFT JOIN 
        `datario.dados_mestres.bairro` AS b
    ON 
        c.id_bairro = b.id_bairro
    WHERE 
        DATE(c.data_inicio) = '2023-04-01'
        AND b.id_bairro IS NULL;
    '''
    df5 = bd.read_sql(query5, billing_project_id)
    return df5

In [80]:
# Retrieve the subprefecture with the highest number of calls for April 1, 2023
answer5 = calls_without_assigned_neighborhood_or_subprefecture('2023-04-01')

Downloading: 100%|██████████| 73/73 [00:00<00:00, 175.11rows/s]


In [81]:
# Show the result
display(answer5)

,id_chamado,data_inicio,id_bairro,bairro_associado
0,18517061,2023-04-01 11:47:17,None,None
1,18517502,2023-04-01 14:34:27,None,None
2,18517263,2023-04-01 12:58:25,None,None
3,18517485,2023-04-01 14:30:06,None,None
4,18517022,2023-04-01 11:34:57,None,None
...,...,...,...,...
68,18516523,2023-04-01 08:58:14,None,None
69,18517483,2023-04-01 14:29:42,None,None
70,18518366,2023-04-01 21:52:56,None,None
71,18517314,2023-04-01 13:18:43,None,None


# **Chamados do 1746 em grandes eventos**
## **6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?**

In [82]:
def count_noise_disturbance_calls(start_date: str, end_date: str):
    '''
    Count the number of calls with the subtype "Perturbação do sossego" 
    that were opened between the given start and end dates (inclusive).

    Parameters:
    start_date (str): The start date for the date range in 'YYYY-MM-DD' format.
    end_date (str): The end date for the date range in 'YYYY-MM-DD' format.

    Returns:
    pandas.DataFrame: A DataFrame containing the total number of calls with the specified subtype.
    '''
    
    query6 = f'''
    SELECT 
        COUNT(*) AS total_chamados
    FROM 
        `datario.adm_central_atendimento_1746.chamado`
    WHERE 
        subtipo = 'Perturbação do sossego'
        AND DATE(data_inicio) BETWEEN '{start_date}' AND '{end_date}';
    '''
    df6 = bd.read_sql(query6, billing_project_id)
    return df6

In [83]:
# Retrieve the total number of calls with the subtype "Perturbação do sossego" 
# opened from 01/01/2022 to 31/12/2023
answer6 = count_noise_disturbance_calls('2022-01-01', '2023-12-31')

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.26rows/s]


In [84]:
# Show the result
display(answer6)

,total_chamados
0,42830


## **7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).**

In [88]:
def subtype_events_calls(start_date: str, end_date: str):
    '''
    Retrieve details of calls with the subtype "Perturbação do sossego" that occurred during specific events
    between the specified start and end dates. The events considered are "Reveillon", "Carnaval", and "Rock in Rio".
    
    Parameters:
    start_date (str): The start date for the date range in 'YYYY-MM-DD' format.
    end_date (str): The end date for the date range in 'YYYY-MM-DD' format.
    
    Returns:
    pandas.DataFrame: A DataFrame containing the ID of the calls, the start date of the calls, the event name, 
    the start date of the event, and the end date of the event.
    '''
    query7 = f'''
    SELECT 
        c.id_chamado,
        c.data_inicio,
        e.evento,
        e.data_inicial AS data_inicio_evento,
        e.data_final AS data_fim_evento
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    JOIN 
        `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON 
        DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
    WHERE 
        c.subtipo = 'Perturbação do sossego'
        AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio');
    '''
    df7 = bd.read_sql(query7, billing_project_id)
    return df7

In [91]:
# Retrieve details of calls with the subtype "Perturbação do sossego" 
# that occurred during specific events between the specified start and end dates.
answer7 = subtype_events_calls('2022-01-01', '2023-12-31')

Downloading: 100%|██████████| 1214/1214 [00:00<00:00, 1328.20rows/s]


In [92]:
# Show the result
display(answer7)

,id_chamado,data_inicio,evento,data_inicio_evento,data_fim_evento
0,17684038,2022-09-10 18:54:32,Rock in Rio,2022-09-08,2022-09-11
1,17661605,2022-09-02 21:17:23,Rock in Rio,2022-09-02,2022-09-04
2,17683780,2022-09-10 16:05:02,Rock in Rio,2022-09-08,2022-09-11
3,17661731,2022-09-02 23:20:51,Rock in Rio,2022-09-02,2022-09-04
4,17661721,2022-09-02 23:04:39,Rock in Rio,2022-09-02,2022-09-04
...,...,...,...,...,...
1209,18080145,2023-01-01 17:59:13,Reveillon,2022-12-30,2023-01-01
1210,18079544,2022-12-31 21:37:09,Reveillon,2022-12-30,2023-01-01
1211,18079154,2022-12-31 14:03:28,Reveillon,2022-12-30,2023-01-01
1212,18075963,2022-12-30 03:20:28,Reveillon,2022-12-30,2023-01-01



## **8. Quantos chamados desse subtipo foram abertos em cada evento?**

In [93]:
def count_noise_disturbance_calls_per_event(start_date: str, end_date: str):
    '''
    Retrieve the total number of calls for the "Noise Disturbance" subtype
    associated with specific events within a given date range.
    
    This function queries the database to find the total number of calls 
    categorized as "Noise Disturbance" for each specified event ('Reveillon', 
    'Carnaval', 'Rock in Rio') that occurred between the provided start and 
    end dates. It returns the total calls per event.
    
    Parameters:
        start_date (str): The start date of the range to search for calls, 
                          in 'YYYY-MM-DD' format.
        end_date (str): The end date of the range to search for calls, 
                        in 'YYYY-MM-DD' format.
    
    Returns:
        DataFrame: A DataFrame containing the event names and the corresponding
                   total number of noise disturbance calls for each event.
    '''

    query8 = f'''
    SELECT 
        e.evento,
        COUNT(c.id_chamado) AS total_chamados_evento
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    JOIN 
        `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON 
        DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
    WHERE 
        c.subtipo = 'Perturbação do sossego'
        AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
    GROUP BY 
        e.evento;
    '''
    df8 = bd.read_sql(query8, billing_project_id)
    return df8

In [94]:
# Retrieve the total number of calls for the "Noise Disturbance" subtype associated 
# with specific events between the specified start and end dates
answer8 = count_noise_disturbance_calls_per_event('2022-01-01', '2023-12-31')

Downloading: 100%|██████████| 3/3 [00:00<00:00,  7.46rows/s]


In [95]:
# Show the result
display(answer8)

,evento,total_chamados_evento
0,Rock in Rio,834
1,Carnaval,241
2,Reveillon,139


## **9. Qual evento teve a maior média diária de chamados abertos desse subtipo?**

In [96]:
def event_w_highest_daily_avg_calls(start_date: str, end_date: str):
    '''
    Find the event with the highest daily average of noise disturbance calls.

    This function retrieves the event among 'Reveillon', 'Carnaval', and 'Rock in Rio'
    that has the highest daily average number of calls for the noise disturbance subtype.
    It calculates the average based on the number of calls per day within the given event
    period.

    Parameters:
        start_date (str): The start date of the period to consider (inclusive) in 'YYYY-MM-DD' format.
        end_date (str): The end date of the period to consider (inclusive) in 'YYYY-MM-DD' format.

    Returns:
        DataFrame: A DataFrame containing the event name and its highest daily average calls.
    '''
    
    query9 = f'''
    SELECT 
        e.evento,
        ROUND(COUNT(c.id_chamado) / COUNT(DISTINCT DATE(c.data_inicio)), 2) AS media_diaria_chamados
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    JOIN 
        `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON 
        DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
    WHERE 
        c.subtipo = 'Perturbação do sossego'
        AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
    GROUP BY 
        e.evento
    ORDER BY 
        media_diaria_chamados DESC
    LIMIT 1;
    '''
    df9 = bd.read_sql(query9, billing_project_id)
    return df9

In [97]:
# Find the event with the highest daily average of noise disturbance calls
answer9 = event_w_highest_daily_avg_calls('2022-01-01', '2023-12-31')

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.43rows/s]


In [98]:
# Show the result
display(answer9)

,evento,media_diaria_chamados
0,Rock in Rio,119.14


## **10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.**

In [102]:
def daily_avg_calls_within_and_outside_events(start_date: str, end_date: str):
    '''
    Compare daily average noise disturbance calls during specific events and the overall period.

    This function calculates and compares the daily average number of noise disturbance calls 
    during the 'Reveillon', 'Carnaval', and 'Rock in Rio' events with the daily average for 
    the entire period from the specified start date to the end date. It uses SQL queries to 
    aggregate the number of calls and days for each event and the total period.

    Parameters:
        start_date (str): The start date of the period to consider (inclusive) in 'YYYY-MM-DD' format.
        end_date (str): The end date of the period to consider (inclusive) in 'YYYY-MM-DD' format.

    Returns:
        DataFrame: A DataFrame containing the period description ('Total Geral' for the overall period 
                   and event names for the specific events) and their corresponding daily average calls.
    '''
    query10 = '''
    WITH total_chamados AS (
    SELECT 
        COUNT(*) AS total_chamados,
        COUNT(DISTINCT DATE(data_inicio)) AS total_dias
    FROM 
        `datario.adm_central_atendimento_1746.chamado`
    WHERE 
        subtipo = 'Perturbação do sossego'
        AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
),
eventos_chamados AS (
    SELECT 
        e.evento,
        COUNT(c.id_chamado) AS total_chamados_evento,
        COUNT(DISTINCT DATE(c.data_inicio)) AS total_dias_evento
    FROM 
        `datario.adm_central_atendimento_1746.chamado` AS c
    JOIN 
        `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON 
        DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
    WHERE 
        c.subtipo = 'Perturbação do sossego'
        AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
    GROUP BY 
        e.evento
)
SELECT 
    'Total Geral' AS periodo,
    ROUND(total_chamados.total_chamados / total_chamados.total_dias, 2) AS media_diaria
FROM 
    total_chamados
UNION ALL
SELECT 
    evento,
    ROUND(total_chamados_evento / total_dias_evento, 2) AS media_diaria
FROM 
    eventos_chamados;
    '''

    df10 = bd.read_sql(query10, billing_project_id)
    return df10

In [103]:
# Calculate and compare the daily average of noise disturbance calls during specific events
# ('Reveillon', 'Carnaval', 'Rock in Rio') with the overall daily average for the entire period 
# from January 1, 2022, to December 31, 2023
answer10 = daily_avg_calls_within_and_outside_events('2022-01-01', '2023-12-31')

Downloading: 100%|██████████| 4/4 [00:00<00:00, 10.60rows/s]


In [101]:
# Show the result
display(answer10)

,periodo,media_diaria
0,Total Geral,61.98
1,Carnaval,60.25
2,Rock in Rio,119.14
3,Reveillon,46.33
